# Homework 2 - Wikipedia Web Traffic Time Series

У вас є дані про відвідування 1000 сторінок Вікіпедії з різних країн та різних девайсів (дані взяті з [Kaggle змагання](https://www.kaggle.com/c/web-traffic-time-series-forecasting))

*wikipedia_train* и *wikipedia_test* - містять дані про трафік. Це файли CSV, де кожен рядок відповідає певній статті, і кожен стовпець відповідає конкретній даті. У деяких записах дані відсутні. Назви сторінок містять проект Вікіпедії (наприклад, en.wikipedia.org), тип доступу (наприклад, desktop) та тип агента (наприклад, spider). Кожне ім'я статті має такий формат: «name_project_access_agent» (наприклад, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам треба відповісти на [питання](https://forms.gle/tx4gSVpCn6cvHMYY6) і спробувати зробити найпростішу модель, яка зможе передбачати майбутні відвідування.

Ось приклади часових рядів відвідуваності сторінок Вікіпедії (*сині* - навчальна вибірка, *зелені* - передбачення моделі переможця змагання на Kaggle, *помаранчеві* - реальні значення):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [100]:
import re
import pandas as pd
import numpy as np

In [101]:
train = pd.read_csv("../data/wikipedia_train.csv")
test = pd.read_csv("../data/wikipedia_test.csv")

train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31
0,15._November_de.wikipedia.org_desktop_all-agents,32.0,26.0,22.0,22.0,29.0,49.0,20.0,27.0,19.0,...,29.0,23.0,31.0,25.0,27.0,23.0,17.0,26.0,23.0,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2.0,3.0,5.0,3.0,5.0,3.0,7.0,8.0,7.0,...,5.0,5.0,6.0,5.0,4.0,11.0,2.0,0.0,7.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,1.0,3.0,2.0,2.0,1.0,10.0,2.0,1.0,4.0,...,4.0,3.0,2.0,3.0,2.0,4.0,2.0,0.0,5.0,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,3.0,3.0,3.0,8.0,12.0,12.0,8.0,12.0,23.0,...,10.0,14.0,26.0,5.0,29.0,23.0,17.0,16.0,12.0,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,24.0,40.0,23.0,49.0,88.0,25.0,31.0,76.0,51.0,...,134.0,162.0,208.0,179.0,108.0,99.0,49.0,80.0,113.0,173.0


## Data Analysis

In [102]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

In [103]:
pages = train["Page"]
pages

0       15._November_de.wikipedia.org_desktop_all-agents
1         2012_(film)_fr.wikipedia.org_all-access_spider
2      2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...
3      2016_UEFA_Europa_League_Final_en.wikipedia.org...
4      2016_in_video_gaming_en.wikipedia.org_all-acce...
                             ...                        
934           黃義雄_zh.wikipedia.org_all-access_all-agents
935            黄渤_zh.wikipedia.org_all-access_all-agents
936           黑嘉嘉_zh.wikipedia.org_all-access_all-agents
937               黒木瞳_ja.wikipedia.org_all-access_spider
938            齊秦_zh.wikipedia.org_mobile-web_all-agents
Name: Page, Length: 939, dtype: object

In [104]:
# Скільки сторінок з французької Вікіпедії у датасеті?
fr_pages = pages.apply(lambda page: page if get_language(page) == 'fr' else None)
fr_pages = fr_pages[~fr_pages.isnull()]
fr_pages_number = fr_pages.count()
print(fr_pages_number)

128


In [105]:
# Яка найпопулярніша сторінка російської Вікіпедії (в середньому) на train?
ru_df = train[train['Page'].str.contains('_ru.')]
# print(ru_df)
ru_df["Total Visits"] = ru_df.sum(axis=1)
ru_df
most_popular_ru_page = ru_df.loc[ru_df["Total Visits"].idxmax()]
print(most_popular_ru_page)

Page            Facebook_ru.wikipedia.org_desktop_all-agents
2015-07-01                                            1665.0
2015-07-02                                            1419.0
2015-07-03                                            1440.0
2015-07-04                                            1062.0
                                    ...                     
2016-08-28                                            1318.0
2016-08-29                                            1708.0
2016-08-30                                            1748.0
2016-08-31                                            2235.0
Total Visits                                        929291.0
Name: 180, Length: 430, dtype: object


/var/folders/06/zr_njqz51_j3yrt1l7_8wvpc0000gn/T/ipykernel_92354/1264956979.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ru_df["Total Visits"] = ru_df.sum(axis=1)
/var/folders/06/zr_njqz51_j3yrt1l7_8wvpc0000gn/T/ipykernel_92354/1264956979.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ru_df["Total Visits"] = ru_df.sum(axis=1)


## Forecasting

Потрібно перетворитиь `train` дані в наступний формат:

In [111]:
train.head()

,Page,date,Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-08-31,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-08-31,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-08-31,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-08-31,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-08-31,173.0
...,...,...,...
934,黃義雄_zh.wikipedia.org_all-access_all-agents,2016-08-31,20.0
935,黄渤_zh.wikipedia.org_all-access_all-agents,2016-08-31,414.0
936,黑嘉嘉_zh.wikipedia.org_all-access_all-agents,2016-08-31,360.0
937,黒木瞳_ja.wikipedia.org_all-access_spider,2016-08-31,304.0


In [5]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31
0,15._November_de.wikipedia.org_desktop_all-agents,32.0,26.0,22.0,22.0,29.0,49.0,20.0,27.0,19.0,...,29.0,23.0,31.0,25.0,27.0,23.0,17.0,26.0,23.0,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2.0,3.0,5.0,3.0,5.0,3.0,7.0,8.0,7.0,...,5.0,5.0,6.0,5.0,4.0,11.0,2.0,0.0,7.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,1.0,3.0,2.0,2.0,1.0,10.0,2.0,1.0,4.0,...,4.0,3.0,2.0,3.0,2.0,4.0,2.0,0.0,5.0,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,3.0,3.0,3.0,8.0,12.0,12.0,8.0,12.0,23.0,...,10.0,14.0,26.0,5.0,29.0,23.0,17.0,16.0,12.0,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,24.0,40.0,23.0,49.0,88.0,25.0,31.0,76.0,51.0,...,134.0,162.0,208.0,179.0,108.0,99.0,49.0,80.0,113.0,173.0


Таким чином у вас кожен рядок містить набір фіч (`Page`, `date`) та цільову змінну (`Visits`). Перетворити дані на такий формат допоможе функція `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оцінювати якість передбачень ми будемо за допомогою [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [161]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Щоб оцінити точність прогнозу потрібно перетворити `test` дані в наступний формат:

In [ ]:
prediction.head()

,Page,date,Visits,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43,37
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6,5
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4,4
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15,14
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101,173


І потім викликати функцію для підрахунку метрики:

In [ ]:
pandas_smape(prediction)

### Last day baseline

Потрібно зробити прогноз на основі відвідувань в останній відомий нам день з train (продублювати значення для кожного дня у test)

In [171]:
train_df = pd.melt(frame=train, id_vars="Page", value_vars=["2016-08-31"], value_name="Visits", var_name="date")
train_df

,Page,date,Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-08-31,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-08-31,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-08-31,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-08-31,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-08-31,173.0
...,...,...,...
934,黃義雄_zh.wikipedia.org_all-access_all-agents,2016-08-31,20.0
935,黄渤_zh.wikipedia.org_all-access_all-agents,2016-08-31,414.0
936,黑嘉嘉_zh.wikipedia.org_all-access_all-agents,2016-08-31,360.0
937,黒木瞳_ja.wikipedia.org_all-access_spider,2016-08-31,304.0


In [172]:
prediction_df = pd.melt(frame=test, id_vars="Page", value_vars=["2016-09-10"], value_name="Visits", var_name="date")
prediction_df["pred_Visits"] = train_df["Visits"]
prediction_df["Visits"].fillna(0, inplace=True)
prediction_df["pred_Visits"].fillna(0, inplace=True)
prediction_df["Visits"] = prediction_df["Visits"].astype(int)
prediction_df["pred_Visits"] = prediction_df["pred_Visits"].astype(int)
prediction_df.head()

,Page,date,Visits,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43,37
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6,5
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4,4
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15,14
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101,173


In [173]:
pandas_smape(prediction_df)

48.89974756385376

In [174]:
prediction_df

,Page,date,Visits,pred_Visits,SMAPE
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43,37,15.000000
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6,5,18.181818
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4,4,0.000000
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15,14,6.896552
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101,173,52.554745
...,...,...,...,...,...
934,黃義雄_zh.wikipedia.org_all-access_all-agents,2016-09-10,2406,20,196.702391
935,黄渤_zh.wikipedia.org_all-access_all-agents,2016-09-10,446,414,7.441860
936,黑嘉嘉_zh.wikipedia.org_all-access_all-agents,2016-09-10,92,360,118.584071
937,黒木瞳_ja.wikipedia.org_all-access_spider,2016-09-10,59,304,134.986226


### Median baseline

Потрібно зробити прогноз на основі медіани за останні **30** днів з `train`. 

А потім покращити передбачення використовуючи інформацію вихідний це чи ні (скористайтеся функцією [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) та різні вікна для підрахунку медіани (7 днів, 60 днів тощо)

Вам допоможе функція `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

In [211]:
# median - n days
days_slice_dict = {
    7: {
        "first": 422,
        "last": 429,
    },
    30: {
        "first": 399,
        "last": 429,
    },
    60: {
        "first": 369,
        "last": 429,
    },
    100: {
        "first": 329,
        "last": 429,
    },
}
def forecast_by_n_days(days_num):
    train = pd.read_csv("../data/wikipedia_train.csv")
    last_n_train_df = train.iloc[:, days_slice_dict[days_num]["first"]:days_slice_dict[days_num]["last"]]
    last_n_train_df["Median_Visits"] = last_n_train_df.median(axis=1)
    prediction_n_df = pd.melt(frame=test, id_vars="Page", value_vars=["2016-09-10"], value_name="Visits", var_name="date")
    prediction_n_df["pred_Visits"] = last_n_train_df["Median_Visits"]
    prediction_n_df["Visits"].fillna(0, inplace=True)
    prediction_n_df["pred_Visits"].fillna(0, inplace=True)
    prediction_n_df["Visits"] = prediction_n_df["Visits"].astype(int)
    prediction_n_df["pred_Visits"] = prediction_n_df["pred_Visits"].astype(int)
    return pandas_smape(prediction_n_df)

In [195]:
# median - 30 days
print(forecast_by_n_days(30))

42.82978095043209


In [197]:
# median - 7 days
print(forecast_by_n_days(7))

42.345140356146246


In [200]:
# median - 60 days
print(forecast_by_n_days(60))

42.63735494702768


In [212]:
# median - 100 days
print(forecast_by_n_days(100))

43.66553680815694
